In [1]:
from sklearn.metrics import classification_report

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import json

# Paso 1: Carga el dataset
df = pd.read_csv('dataset_preprocesado.csv')  


print("Columnas del dataset:", df.columns.tolist())
print(df.head())

# Paso 2: Identifica la columna objetivo
target_column = 'Churn'  
X = df.drop(columns=[target_column])  # Características
y = df[target_column]  # Objetivo

# Verifica los valores de la columna objetivo
print("Valores de la columna objetivo:", y.value_counts())

# Paso 3: Divide los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)
print(X_train.dtypes)
print(X_train.head())

# Paso 4: Preprocesamiento
categorical_columns = X_train.select_dtypes(include=['object']).columns
numeric_columns = X_train.select_dtypes(include=['int64', 'float64']).columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_columns),
        ('num', 'passthrough', numeric_columns)
    ]
)

# Paso 5: Random Forest
rf = RandomForestClassifier()
pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', rf)
])
param_grid_rf = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [10, 20, None],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}
grid_search_rf = GridSearchCV(estimator=pipeline_rf, param_grid=param_grid_rf, cv=5, scoring='f1', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)
best_rf = grid_search_rf.best_estimator_
print("Mejores hiperparámetros RF:", grid_search_rf.best_params_)
print("Mejor F1-score (validación) RF:", grid_search_rf.best_score_)

# Paso 6: XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
pipeline_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb)
])
param_grid_xgb = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [3, 6, 9],
    'classifier__learning_rate': [0.01, 0.1, 0.3]
}
grid_search_xgb = GridSearchCV(estimator=pipeline_xgb, param_grid=param_grid_xgb, cv=5, scoring='f1', n_jobs=-1)
grid_search_xgb.fit(X_train, y_train)
print("Mejores hiperparámetros XGB:", grid_search_xgb.best_params_)
print("Mejor F1-score (validación) XGB:", grid_search_xgb.best_score_)

# Paso 7: Predicciones y comparación
y_pred_rf = best_rf.predict(X_test)
y_pred_xgb = grid_search_xgb.best_estimator_.predict(X_test)
print("Random Forest:\n", classification_report(y_test, y_pred_rf))
print("XGBoost:\n", classification_report(y_test, y_pred_xgb))

# Paso 8: Guarda resultados en JSON
report_rf = classification_report(y_test, y_pred_rf, output_dict=True)
report_xgb = classification_report(y_test, y_pred_xgb, output_dict=True)
resultados = {
    "random_forest": {
        "reporte": report_rf,
        "mejores_hiperparametros": grid_search_rf.best_params_,
        "mejor_f1_score": grid_search_rf.best_score_
    },
    "xgboost": {
        "reporte": report_xgb,
        "mejores_hiperparametros": grid_search_xgb.best_params_,
        "mejor_f1_score": grid_search_xgb.best_score_
    }
}
with open('comparacion_modelos.json', 'w', encoding='utf-8') as archivo:
    json.dump(resultados, archivo, indent=4, ensure_ascii=False)
print("Resultados guardados en comparacion_modelos.json")

Columnas del dataset: ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 'Churn', 'gender_Male', 'Partner_Yes', 'Dependents_Yes', 'PhoneService_Yes', 'MultipleLines_No phone service', 'MultipleLines_Yes', 'InternetService_Fiber optic', 'InternetService_No', 'OnlineSecurity_No internet service', 'OnlineSecurity_Yes', 'OnlineBackup_No internet service', 'OnlineBackup_Yes', 'DeviceProtection_No internet service', 'DeviceProtection_Yes', 'TechSupport_No internet service', 'TechSupport_Yes', 'StreamingTV_No internet service', 'StreamingTV_Yes', 'StreamingMovies_No internet service', 'StreamingMovies_Yes', 'Contract_One year', 'Contract_Two year', 'PaperlessBilling_Yes', 'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check']
   SeniorCitizen    tenure  MonthlyCharges  TotalCharges  Churn  gender_Male  \
0              0 -1.277445       -1.160323     -0.994242      0        False   
1              0  0.066327       -0.259629     -0.

Mejores hiperparámetros RF: {'classifier__max_depth': 10, 'classifier__min_samples_leaf': 4, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 100}
Mejor F1-score (validación) RF: 0.5280654681460594


/Users/anasofiagarcia/anaconda3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:00:53] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/anasofiagarcia/anaconda3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:00:53] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/anasofiagarcia/anaconda3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:00:53] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/anasofiagarcia/anaconda3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:00:53] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "

/Users/anasofiagarcia/anaconda3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:00:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/anasofiagarcia/anaconda3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:00:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/anasofiagarcia/anaconda3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:00:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/anasofiagarcia/anaconda3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:00:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "

/Users/anasofiagarcia/anaconda3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:01:12] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/anasofiagarcia/anaconda3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:01:12] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/anasofiagarcia/anaconda3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:01:12] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/anasofiagarcia/anaconda3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:01:12] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "

/Users/anasofiagarcia/anaconda3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:01:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/anasofiagarcia/anaconda3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:01:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/anasofiagarcia/anaconda3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:01:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/anasofiagarcia/anaconda3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:01:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "

Mejores hiperparámetros XGB: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 100}
Mejor F1-score (validación) XGB: 0.5340928815537839
Random Forest:
               precision    recall  f1-score   support

           0       0.83      0.92      0.87      1036
           1       0.69      0.47      0.56       373

    accuracy                           0.80      1409
   macro avg       0.76      0.70      0.71      1409
weighted avg       0.79      0.80      0.79      1409

XGBoost:
               precision    recall  f1-score   support

           0       0.83      0.92      0.88      1036
           1       0.70      0.48      0.57       373

    accuracy                           0.81      1409
   macro avg       0.76      0.70      0.72      1409
weighted avg       0.80      0.81      0.80      1409

Resultados guardados en comparacion_modelos.json
